In [1]:
from tinygrad.densetensor import DenseTensor
from tinygrad.sparsetensor import SparseTensor
import numpy as np
import tinygrad.optim as optim
from extra.utils import get_parameters
from test.test_mnist import fetch_mnist
from tqdm import trange

%load_ext autoreload
%autoreload 2

DEVICE:GPU


In [2]:
dim1 = 32
dim2 = 784
dim3 = 10

np.random.seed(9)

seedmat = np.random.randn(dim2,dim3).astype(np.float32) / 1000

In [3]:
X_train, Y_train, X_test, Y_test = fetch_mnist()
X_train.shape

(60000, 784)

In [4]:
x_init = np.random.randn(dim1,dim2).astype(np.float32)
x = DenseTensor(x_init)
y = DenseTensor(np.random.randn(dim1,dim3))

warning, (32, 10) isn't float32, it's float64


In [5]:
class MLP:
  def __init__(self):
    self.W = DenseTensor(seedmat)

  def parameters(self):
    return get_parameters(self)

  def forward(self, x):
    out = x.dot(self.W)
    out = out.logsoftmax()
    #out = out.mul(self.m).add(self.m).sum()
    return out

In [6]:
class MLP2:
  def __init__(self):
    self.W = SparseTensor(seedmat)

  def parameters(self):
    return get_parameters(self)

  def forward(self, x):
    out = x.dot(self.W)
    out = out.logsoftmax()
    #out = out.mul(self.m).add(self.m).sum()
    return out

In [7]:
def loss_fn(y, y_pred):
    return ((y-y_pred)**2)**.5

In [8]:
def sparse_categorical_crossentropy(out, Y):
  num_classes = out.shape[-1]
  YY = Y.flatten()
  y = np.zeros((YY.shape[0], num_classes), np.float32)
  # correct loss for NLL, torch NLL loss returns one per row
  y[range(y.shape[0]),YY] = -1.0*num_classes
  y = y.reshape(list(Y.shape)+[num_classes])
  y = DenseTensor(y)
  return out.mul(y).mean()

### Dense

In [9]:
model = MLP()

In [10]:
optimizer = optim.SGD(model.parameters(), lr=.0001)

In [11]:
X_train.shape, Y_train.shape

((60000, 784), (60000,))

In [12]:
loss_fn = sparse_categorical_crossentropy

In [13]:
BS = dim1
epochs = 1

for epoch in range(epochs):
    losses = []
    accs = []
    for i in trange(int(X_train.shape[0]/BS)):
        x = DenseTensor(X_train[i*BS:(i+1)*BS])
        y = Y_train[i*BS:(i+1)*BS]
        optimizer.zero_grad()
        res = model.forward(x)
        loss = loss_fn(res, y)
        #print(res.cpu().data[:4])
        #print(y[:4])
        loss.backward()
        optimizer.step()
        
        cat = np.argmax(res.cpu().data, axis=-1)
        accuracy = (cat == y).mean()
        
        losses.append(loss.cpu().data)
        accs.append(accuracy)
        if i % 1 == 0:
            print("LOSS:%.2f  ACC:%.2f" % (np.array(losses).mean(), np.array(accs).mean()))
            accs, losses = [], []  
            
    losses = []
    accs = []
    for i in trange(int(X_test.shape[0]/BS)):
        x = DenseTensor(X_test[i*BS:(i+1)*BS])
        y = Y_test[i*BS:(i+1)*BS]
        res = model.forward(x)
        loss = loss_fn(res, y)
        
        cat = np.argmax(res.cpu().data, axis=-1)
        accuracy = (cat == y).mean()
        
        losses.append(loss.cpu().data)
        accs.append(accuracy)
        if i % 32 == 0:
            print("TEST_LOSS:%.2f  TEST_ACC:%.2f" % (np.array(losses).mean(), np.array(accs).mean()))
            accs, losses = [], []

  1%|█▌                                                                                                                                                    | 19/1875 [00:00<00:19, 94.16it/s]

LOSS:4.38  ACC:0.03
LOSS:39.29  ACC:0.19
LOSS:31.43  ACC:0.19
LOSS:28.64  ACC:0.69
LOSS:37.58  ACC:0.28
LOSS:49.48  ACC:0.44
LOSS:8.22  ACC:0.59
LOSS:13.19  ACC:0.41
LOSS:12.42  ACC:0.47
LOSS:7.54  ACC:0.75
LOSS:6.04  ACC:0.72
LOSS:7.32  ACC:0.59
LOSS:8.17  ACC:0.62
LOSS:10.00  ACC:0.31
LOSS:14.23  ACC:0.62
LOSS:5.23  ACC:0.59
LOSS:13.33  ACC:0.56
LOSS:14.40  ACC:0.41
LOSS:14.08  ACC:0.53
LOSS:23.96  ACC:0.56
LOSS:5.58  ACC:0.72
LOSS:4.85  ACC:0.78
LOSS:7.03  ACC:0.75


  2%|███▏                                                                                                                                                  | 40/1875 [00:00<00:18, 98.09it/s]

LOSS:2.41  ACC:0.81
LOSS:3.54  ACC:0.69
LOSS:3.34  ACC:0.75
LOSS:6.65  ACC:0.75
LOSS:3.66  ACC:0.69
LOSS:8.05  ACC:0.72
LOSS:3.70  ACC:0.75
LOSS:3.78  ACC:0.72
LOSS:5.13  ACC:0.78
LOSS:8.47  ACC:0.69
LOSS:6.55  ACC:0.75
LOSS:5.76  ACC:0.75
LOSS:7.51  ACC:0.59
LOSS:2.66  ACC:0.81
LOSS:4.88  ACC:0.81
LOSS:8.46  ACC:0.72
LOSS:30.91  ACC:0.16
LOSS:49.37  ACC:0.25
LOSS:39.95  ACC:0.66
LOSS:41.52  ACC:0.50


  3%|████▉                                                                                                                                                | 62/1875 [00:00<00:17, 104.70it/s]

...

 87%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                   | 272/312 [00:00<00:00, 262.13it/s]

TEST_LOSS:1.97  TEST_ACC:0.93


 96%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊      | 299/312 [00:01<00:00, 261.52it/s]

TEST_LOSS:1.21  TEST_ACC:0.95


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 312/312 [00:01<00:00, 291.37it/s]


In [14]:
model.parameters()

[<DenseTensor <GPUBuffer with shape (784, 10)> with grad <GPUBuffer with shape (784, 10)>>,
 <DenseTensor <GPUBuffer with shape (784, 10)> with grad None>]

In [15]:
res.cpu().data

array([[ 0.00000000e+00, -2.79626434e+02, -1.89766403e+02,
        -1.85337097e+02, -2.23227997e+02, -1.08916428e+02,
        -1.45192490e+02, -2.47611206e+02, -1.48810181e+02,
        -2.07967789e+02],
       [-2.18757629e-01, -1.05336243e+02, -3.09417057e+01,
        -7.73215179e+01, -6.42391052e+01, -4.75491676e+01,
        -1.62717819e+00, -9.25289993e+01, -3.92931671e+01,
        -4.98421555e+01],
       [-2.59525299e-01, -1.45203674e+02, -1.47586060e+00,
        -2.73228645e+01, -1.28580261e+02, -4.31444931e+01,
        -7.60789566e+01, -6.88039017e+01, -4.62150269e+01,
        -4.64716873e+01],
       [-6.26183167e+01,  0.00000000e+00, -2.62000351e+01,
        -4.95433235e+01, -7.99050140e+01, -8.71161270e+01,
        -8.53950348e+01, -1.19562302e+02, -3.10348167e+01,
        -8.63125458e+01],
       [-1.19111023e+02,  0.00000000e+00, -2.46872978e+01,
        -5.81748123e+01, -8.36772308e+01, -1.09548973e+02,
        -8.57932205e+01, -9.40537186e+01, -5.28003159e+01,
        -9.

In [16]:
loss.cpu().data

array([4.969031], dtype=float32)

In [17]:
model.W.grad.cpu().data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
 model.W.cpu().data

array([[ 1.1085547e-06, -2.8954408e-04, -1.1160664e-03, ...,
        -4.9087201e-04, -2.4068057e-04, -6.4794743e-04],
       [ 6.3589111e-04,  1.7401173e-03,  2.9668221e-04, ...,
        -9.0072118e-04, -1.3712501e-04,  1.2975790e-03],
       [ 6.7527115e-04,  3.1958120e-05,  9.1814587e-04, ...,
         3.2841109e-04, -4.9822475e-04, -2.0917768e-03],
       ...,
       [ 9.3478512e-04, -1.8844857e-04,  7.9919404e-04, ...,
         2.3419906e-04,  1.0647394e-03,  8.2097884e-04],
       [-1.4317269e-03,  5.1616301e-04,  5.3125765e-04, ...,
         3.0876859e-04,  7.2942115e-04, -1.3654447e-03],
       [-3.5375252e-04,  2.5481975e-03,  1.0601099e-03, ...,
        -8.8676868e-04, -1.2762052e-03,  1.6803305e-03]], dtype=float32)

### Second

In [19]:
model2 = MLP2()

In [20]:
optimizer2 = optim.SGD(model2.parameters(), lr=.0001)

In [22]:
BS = dim1
epochs = 10

for epoch in range(epochs):
    losses = []
    accs = []
    for i in trange(int(X_train.shape[0]/BS)):
        x = DenseTensor(X_train[i*BS:(i+1)*BS])
        y = Y_train[i*BS:(i+1)*BS]
        optimizer2.zero_grad()
        res2 = model2.forward(x)
        loss2 = loss_fn(res2, y)
        #print(res2.cpu().data[:4])
        #print(y[:4])
        loss2.backward()
        optimizer2.step()
        
        cat = np.argmax(res2.cpu().data, axis=-1)
        accuracy = (cat == y).mean()
        
        losses.append(loss2.cpu().data)
        accs.append(accuracy)
        if i % 1 == 0:
            print("LOSS:%.2f  ACC:%.2f" % (np.array(losses).mean(), np.array(accs).mean()))
            accs, losses = [], []  
            
    losses = []
    accs = []
    for i in trange(int(X_test.shape[0]/BS)):
        x = DenseTensor(X_test[i*BS:(i+1)*BS])
        y = Y_test[i*BS:(i+1)*BS]
        res2 = model2.forward(x)
        loss2 = loss_fn(res2, y)
        
        cat = np.argmax(res2.cpu().data, axis=-1)
        accuracy = (cat == y).mean()
        
        losses.append(loss2.cpu().data)
        accs.append(accuracy)
        if i % 32 == 0:
            print("TEST_LOSS:%.2f  TEST_ACC:%.2f" % (np.array(losses).mean(), np.array(accs).mean()))
            accs, losses = [], []

  1%|█▎                                                                                                                                                    | 17/1875 [00:00<00:22, 82.14it/s]

LOSS:4.25  ACC:0.06
LOSS:3.67  ACC:0.00
LOSS:3.69  ACC:0.12
LOSS:3.38  ACC:0.19
LOSS:3.66  ACC:0.03
LOSS:3.29  ACC:0.12
LOSS:2.89  ACC:0.19
LOSS:3.32  ACC:0.22
LOSS:2.96  ACC:0.22
LOSS:3.06  ACC:0.12
LOSS:3.04  ACC:0.12
LOSS:2.81  ACC:0.19
LOSS:2.85  ACC:0.22
LOSS:2.93  ACC:0.25
LOSS:2.66  ACC:0.22
LOSS:3.32  ACC:0.12
LOSS:3.12  ACC:0.28
LOSS:2.71  ACC:0.22
LOSS:3.05  ACC:0.22


  2%|██▊                                                                                                                                                   | 35/1875 [00:00<00:21, 86.63it/s]

LOSS:3.20  ACC:0.19
LOSS:3.21  ACC:0.12
LOSS:2.89  ACC:0.34
LOSS:3.53  ACC:0.16
LOSS:3.59  ACC:0.25
LOSS:2.81  ACC:0.25
LOSS:2.71  ACC:0.19
LOSS:3.06  ACC:0.22
LOSS:2.93  ACC:0.12
LOSS:3.59  ACC:0.12
LOSS:2.95  ACC:0.09
LOSS:3.16  ACC:0.12
LOSS:2.88  ACC:0.22
LOSS:2.92  ACC:0.28
LOSS:2.45  ACC:0.25
LOSS:2.89  ACC:0.22
LOSS:3.04  ACC:0.19
LOSS:2.66  ACC:0.16


  3%|████▎                                                                                                                                                 | 54/1875 [00:00<00:20, 88.51it/s]

...

 93%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████           | 1737/1875 [00:19<00:01, 91.29it/s]

LOSS:2.35  ACC:0.34
LOSS:2.31  ACC:0.31
LOSS:2.18  ACC:0.25
LOSS:2.34  ACC:0.28
LOSS:2.60  ACC:0.16
LOSS:2.61  ACC:0.22
LOSS:2.42  ACC:0.25
LOSS:2.69  ACC:0.31
LOSS:2.44  ACC:0.22
LOSS:1.90  ACC:0.31
LOSS:2.59  ACC:0.19
LOSS:2.04  ACC:0.50
LOSS:3.34  ACC:0.16
LOSS:2.10  ACC:0.34
LOSS:2.41  ACC:0.19
LOSS:2.57  ACC:0.22
LOSS:2.28  ACC:0.38
LOSS:1.78  ACC:0.38
LOSS:2.19  ACC:0.31


 93%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████          | 1749/1875 [00:20<00:01, 86.94it/s]

LOSS:2.09  ACC:0.28
LOSS:2.19  ACC:0.31
LOSS:2.56  ACC:0.31
LOSS:2.50  ACC:0.22
LOSS:2.54  ACC:0.38
LOSS:2.41  ACC:0.34
LOSS:2.38  ACC:0.25
LOSS:2.57  ACC:0.28
LOSS:2.48  ACC:0.16
LOSS:2.58  ACC:0.34


KeyboardInterrupt: 

In [ ]:
res2.cpu().data==res.cpu().data

In [ ]:
res2.cpu().data

In [ ]:
loss.cpu().data==loss2.cpu().data

In [ ]:
loss2.cpu().data

In [ ]:
res.grad.cpu().data==res2.grad.cpu().data

In [ ]:
res2.grad.cpu().data

In [ ]:
model.W.grad, model2.W.grad

In [ ]:
model.W.grad.cpu().data==model2.W.grad.cpu().data

In [ ]:
model.W.grad.cpu().data

In [ ]:
model2.W.grad.cpu().data

In [ ]:
model.W.grad

In [ ]:
model2.W.cpu().data